In [3]:
from groq import Groq
import os
from dotenv import load_dotenv

from pathlib import Path


dotenv_path=Path('F:\Project\LLMStockPulse-main\.env')

load_dotenv(dotenv_path=dotenv_path)

# Load API key from environment variable
groq_api_key = os.getenv("GROQ_API_KEY")

# Create a client instance
client = Groq(api_key=groq_api_key)

<>:8: SyntaxWarning: invalid escape sequence '\P'
<>:8: SyntaxWarning: invalid escape sequence '\P'
/tmp/ipykernel_3438/2670567023.py:8: SyntaxWarning: invalid escape sequence '\P'
  dotenv_path=Path('F:\Project\LLMStockPulse-main\.env')
/tmp/ipykernel_3438/2670567023.py:8: SyntaxWarning: invalid escape sequence '\P'
  dotenv_path=Path('F:\Project\LLMStockPulse-main\.env')


GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable